In [87]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import re

# Read data

In [2]:
os.getcwd()

'/home/onyxia/scoring_SDA'

In [12]:

# folder path
#dir_path = "C://Users//vnarv//PycharmProjects//scoring_SDA/data"
dir_path = "/home/onyxia/scoring_SDA/"

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
print(res)

['exploratory_analysis.ipynb', 'README.md']


# Explore files 

In [13]:
def check_nan(data):

    nb_lines = data.shape[0]
    nb_columns = data.shape[1]

    print("=== Check NAN")
    nb_nan = data.isna().sum() / nb_lines
    part_nan = nb_nan[nb_nan>0]

    if not part_nan.empty:
        part_nan = part_nan.sort_values(ascending=False)
        nb_cols_nan = len(part_nan.keys())
        print(f"... nb of columns with NAN {nb_cols_nan} out of {nb_columns}")
        for index, value in part_nan.items():
            print(f"{index}... {value:.2%}")

def check_duplicates(data, keys):
    df_dup = data[data.duplicated(subset=keys, keep=False)]
    display(df_dup.head())

In [15]:
for file in res:
    if file.endswith(".csv"):
        t = pd.read_csv(dir_path + file)
        print(f"\n========== File name {file}\n")
        display(t.head())
        print(t.columns)
        check_nan(t)

    elif file.endswith(".xls"):
        # read file
        with open(dir_path + file, mode="rb") as excel_file:
            _excel_file = pd.read_excel(excel_file,)
        print(f"\n========== File name {file}\n")
        print(_excel_file.columns[0])
        _excel_file.set_index(_excel_file.columns[0], inplace=True)
        # drop nan
        _excel_file = _excel_file.dropna(how="all")
        # set header
        header_row = _excel_file.iloc[0]
        df = pd.DataFrame(_excel_file.values[1:], columns=header_row)
        check_nan(df)
        display(df.head())
        print("======== Check NAN")
        

    else:
        print("Check files in data folder")

Check files in data folder
Check files in data folder


### Target 

In [7]:
input_y = "2020_US_County_Level_Presidential_Results.csv"

tmp = pd.read_csv(dir_path + input_y)
tmp.columns

Index(['state_name', 'county_fips', 'county_name', 'votes_gop', 'votes_dem',
       'total_votes', 'diff', 'per_gop', 'per_dem', 'per_point_diff'],
      dtype='object')

In [8]:
tmp.head()

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022


In [32]:
pct_votes_party = tmp[["votes_gop", "votes_dem"]].apply(np.sum) / tmp["total_votes"].sum()
pct_votes_party = pct_votes_party*100

In [33]:
fig = px.bar(pct_votes_party,
             x=pct_votes_party.keys(),
             y=pct_votes_party.values,
             title="2020 U.S. percentage votes per party", 
             labels={"y": "%",
                     "state_name": "State",
                     "index": "Party"},
             color=pct_votes_party.keys(),
             height=400,
             width=800
            )
fig

In [9]:
tmp[tmp["state_name"]=="Delaware"]

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
327,Delaware,10001,Kent County,41009,44552,87025,-3543,0.471232,0.511945,-0.040712
328,Delaware,10003,New Castle County,88364,195034,287633,-106670,0.307211,0.678065,-0.370855
329,Delaware,10005,Sussex County,71230,56682,129352,14548,0.550668,0.438200,0.112468


In [34]:
#df_target = tmp[["state_name", "county_fips", "county_name"]]
tmp["winner_democrats"] = np.where(tmp["votes_gop"] < tmp["votes_dem"], 1, 0) # calculer par state

In [35]:
tmp.head()

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,winner_democrats
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184,0
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623,0
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631,0
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280,0
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022,0


In [73]:
df_target.head()

,state_name,county_fips,county_name,target
0,Alabama,1001,Autauga County,1
1,Alabama,1003,Baldwin County,1
2,Alabama,1005,Barbour County,1
3,Alabama,1007,Bibb County,1
4,Alabama,1009,Blount County,1


In [91]:
tmp.head()
tmp[tmp.duplicated(subset=["state_name", "county_fips", "county_name"], keep=False)]

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff


### Votes per state

In [92]:

total_votes = tmp["total_votes"].sum()
total_republicans = tmp.groupby("state_name")["votes_gop"].sum().reset_index()
total_democrats = tmp.groupby("state_name")["votes_dem"].sum().reset_index()
df = pd.merge(total_republicans, total_democrats, on="state_name", how="inner")
df["total_votes_state"] = df["votes_gop"] + df["votes_dem"]
df["Republican"] = (df["votes_gop"] / df["total_votes_state"])*100
df["Democratic"] = np.abs(100 - df["Republican"])
df = df.sort_values(by="Democratic", ascending=False)
display(df.head())
fig = px.bar(df, x="state_name", y=["Democratic", "Republican"],
             title="U.S. 2020 presidential elections per state", 
             labels={"value": "%",
                     "state_name": "State",
                     "variable": "Party"})
fig.add_hline(y=50, line_width=3, line_dash="dash", line_color="green",
              annotation_text="50%", 
              annotation_position="bottom right")
fig

,state_name,votes_gop,votes_dem,total_votes_state,Republican,Democratic
8,District of Columbia,18586,317323,335909,5.533046,94.466954
45,Vermont,112704,242820,355524,31.700813,68.299187
21,Massachusetts,1167202,2382202,3549404,32.884450,67.115550
20,Maryland,976414,1985023,2961437,32.970953,67.029047
11,Hawaii,196864,366130,562994,34.967335,65.032665


C:\Users\vnarv\AppData\Local\Temp\ipykernel_4892\3698961737.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



votes_gop     1.455063e+06
votes_dem     1.593431e+06
Republican    5.030956e+01
Democratic    4.969044e+01
dtype: float64

In [ ]:
# https://plotly.com/python/mapbox-county-choropleth/ US MAP

In [76]:
top_countys = df.sort_values(
    by="total_votes_state",
    ascending=False
).iloc[:1][["state_name"]]
top_countys.reset_index(inplace=True, drop=True)

In [83]:
top_countys = tmp[tmp["state_name"].isin(top_10_states["state_name"])]
top_countys = top_countys.groupby("county_name").agg({"votes_gop": "sum",
                                             "votes_dem": "sum"})
top_countys["total_votes_county"] = top_countys["votes_gop"] + top_countys["votes_dem"]
top_countys["Democrats"] = (top_countys["votes_gop"] / top_countys["total_votes_county"])*100
top_countys = top_countys.sort_values(by="Democrats", ascending=False)

In [84]:
top_countys.head()

,votes_gop,votes_dem,total_votes_county,Democrats
county_name,,,,
Lassen County,8970,2799,11769,76.217181
Modoc County,3109,1150,4259,72.998356
Tehama County,19141,8911,28052,68.233994
Shasta County,60789,30000,90789,66.956349
Glenn County,7063,3995,11058,63.872310


In [90]:

fig = px.bar(top_countys.reset_index(),x="county_name", y="Democrats",
             title="Pertage votes for democrats for the top countys", 
             labels={"Democrats": "%",
                    "county_name": "County"
                    }
                     )
fig.add_hline(y=50, line_width=3, line_dash="dash", line_color="green",
              annotation_text="50%", 
              annotation_position="bottom right")
fig

In [86]:
# import plotly.graph_objects as go

# fig = go.Figure()

# for col in df:
#     fig.add_trace(go.Box(y=df_top_10_states[per_dem].values, name=df_top_10_states[col].name))
  
# fig.show()

In [103]:
# Tools
def upper_consistent(df):
    df = df.apply(lambda x: x.str.upper() if x.dtype == "object" else x) 
    return df

# Standardise columns names 
def standard_name_cols(data_columns):

    data_columns = data_columns.str.upper()
    col_names = [re.sub(r'[^a-zA-Z0-9]', '_', col) for col in data_columns]

    return col_names


def read_excel(path):
    with open(path, mode="rb") as excel_file:
        _excel_file = pd.read_excel(excel_file, 
        #converters={'FIPS Code': str}
        ) 
    print(f"\n========== File name {file}\n")
    print(_excel_file.columns[0])

    _excel_file.set_index(_excel_file.columns[0], inplace=True)

    # drop nan
    _excel_file = _excel_file.dropna(how="all")
    _excel_file = _excel_file.reset_index()
    
    # set header
    header_row = _excel_file.iloc[0]
    
    df = pd.DataFrame(_excel_file.values[1:], columns=header_row)

    return df

In [ ]:
# next: explorer les fichiers des explicatives et croisser avec y

### Explore target 2020

In [126]:
input_y = "2020_US_County_Level_Presidential_Results.csv"
df_presidential_2020 = pd.read_csv(f"{dir_path}data/{input_y}",
                        converters={'county_fips': str})
df_presidential_2020.columns = standard_name_cols(df_presidential_2020.columns)
df_presidential_2020[["STATE_NAME","COUNTY_NAME"]] = upper_consistent(df_presidential_2020[["STATE_NAME","COUNTY_NAME"]])
df_presidential_2020

,STATE_NAME,COUNTY_FIPS,COUNTY_NAME,VOTES_GOP,VOTES_DEM,TOTAL_VOTES,DIFF,PER_GOP,PER_DEM,PER_POINT_DIFF
0,ALABAMA,01001,AUTAUGA COUNTY,19838,7503,27770,12335,0.714368,0.270184,0.444184
1,ALABAMA,01003,BALDWIN COUNTY,83544,24578,109679,58966,0.761714,0.224090,0.537623
2,ALABAMA,01005,BARBOUR COUNTY,5622,4816,10518,806,0.534512,0.457882,0.076631
3,ALABAMA,01007,BIBB COUNTY,7525,1986,9595,5539,0.784263,0.206983,0.577280
4,ALABAMA,01009,BLOUNT COUNTY,24711,2640,27588,22071,0.895716,0.095694,0.800022
...,...,...,...,...,...,...,...,...,...,...
3147,WYOMING,56037,SWEETWATER COUNTY,12229,3823,16603,8406,0.736554,0.230260,0.506294
3148,WYOMING,56039,TETON COUNTY,4341,9848,14677,-5507,0.295769,0.670982,-0.375213
3149,WYOMING,56041,UINTA COUNTY,7496,1591,9402,5905,0.797277,0.169219,0.628058
3150,WYOMING,56043,WASHAKIE COUNTY,3245,651,4012,2594,0.808824,0.162263,0.646560


In [127]:
df_presidential_2020[df_presidential_2020["COUNTY_NAME"]=="KENT COUNTY"]

,STATE_NAME,COUNTY_FIPS,COUNTY_NAME,VOTES_GOP,VOTES_DEM,TOTAL_VOTES,DIFF,PER_GOP,PER_DEM,PER_POINT_DIFF
327,DELAWARE,10001,KENT COUNTY,41009,44552,87025,-3543,0.471232,0.511945,-0.040712
1216,MARYLAND,24029,KENT COUNTY,5195,5329,10794,-134,0.481286,0.493700,-0.012414
1281,MICHIGAN,26081,KENT COUNTY,165741,187915,361048,-22174,0.459055,0.520471,-0.061416
2322,RHODE ISLAND,44003,KENT COUNTY,42001,49113,93093,-7112,0.451172,0.527569,-0.076397
2664,TEXAS,48263,KENT COUNTY,411,47,462,364,0.889610,0.101732,0.787879


In [179]:
state_list = df_presidential_2020["STATE_NAME"].unique()
county_fips_list = df_presidential_2020["COUNTY_FIPS"].unique()
county_name_list = df_presidential_2020["COUNTY_NAME"].unique()

In [180]:
print(f"Nb of states : {len(state_list)}")
print(f"Nb of county fips : {len(county_fips_list)}")
print(f"Nb of countys : {len(county_name_list)}")

Nb of states : 51
Nb of county fips : 3152
Nb of countys : 1887


# Exploration des variables explicatives

### Population Estimates data


In [133]:
df_population = read_excel(dir_path + "data/PopulationEstimates.xls")
df_population.columns = standard_name_cols(df_population.columns)
df_population[["STATE", "AREA_NAME"]] = upper_consistent(df_population[["STATE", "AREA_NAME"]])
display(df_population.head())
check_nan(df_population)
check_duplicates(df_population, df_population.columns)


========== File name README.md

Population estimates for the U.S., States, and counties, 2010-19 (see the second tab in this workbook for variable name descriptions)


,FIPSTXT,STATE,AREA_NAME,RURAL_URBAN_CONTINUUM_CODE_2003,RURAL_URBAN_CONTINUUM_CODE_2013,URBAN_INFLUENCE_CODE_2003,URBAN_INFLUENCE_CODE_2013,ECONOMIC_TYPOLOGY_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,...,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
0,00000,US,UNITED STATES,NaN,NaN,NaN,NaN,NaN,308745538,308758105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01000,AL,ALABAMA,NaN,NaN,NaN,NaN,NaN,4779736,4780125,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
2,01001,AL,AUTAUGA COUNTY,2,2,2,2,0,54571,54597,...,4.84731,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
3,01003,AL,BALDWIN COUNTY,4,3,5,2,5,182265,182265,...,24.017829,16.64187,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
4,01005,AL,BARBOUR COUNTY,6,6,6,6,3,27457,27455,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664


=== Check NAN
... nb of columns with NAN 150 out of 165
ECONOMIC_TYPOLOGY_2015... 4.00%
R_NET_MIG_2019... 2.44%
R_DEATH_2014... 2.44%
R_NATURAL_INC_2017... 2.44%
R_NATURAL_INC_2016... 2.44%
R_NATURAL_INC_2015... 2.44%
R_NATURAL_INC_2014... 2.44%
R_NATURAL_INC_2013... 2.44%
R_NATURAL_INC_2012... 2.44%
R_DEATH_2019... 2.44%
R_DEATH_2018... 2.44%
R_DEATH_2017... 2.44%
R_DEATH_2016... 2.44%
R_DEATH_2015... 2.44%
R_DEATH_2013... 2.44%
R_NATURAL_INC_2019... 2.44%
R_DEATH_2012... 2.44%
R_DEATH_2011... 2.44%
R_BIRTH_2019... 2.44%
R_BIRTH_2018... 2.44%
R_BIRTH_2017... 2.44%
R_BIRTH_2016... 2.44%
R_BIRTH_2015... 2.44%
R_BIRTH_2014... 2.44%
R_BIRTH_2013... 2.44%
R_BIRTH_2012... 2.44%
R_BIRTH_2011... 2.44%
R_NATURAL_INC_2018... 2.44%
R_NATURAL_INC_2011... 2.44%
R_INTERNATIONAL_MIG_2011... 2.44%
R_DOMESTIC_MIG_2015... 2.44%
R_NET_MIG_2018... 2.44%
R_NET_MIG_2017... 2.44%
R_NET_MIG_2016... 2.44%
R_INTERNATIONAL_MIG_2012... 2.44%
R_NET_MIG_2014... 2.44%
R_NET_MIG_2013... 2.44%
R_NET_MIG_2012... 2.44%

,FIPSTXT,STATE,AREA_NAME,RURAL_URBAN_CONTINUUM_CODE_2003,RURAL_URBAN_CONTINUUM_CODE_2013,URBAN_INFLUENCE_CODE_2003,URBAN_INFLUENCE_CODE_2013,ECONOMIC_TYPOLOGY_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,...,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019


### Education data

Rural-Urban Continuum Codes form a classification scheme that distinguishes metropolitan counties by the population size of their metro area, and nonmetropolitan counties by degree of urbanization and adjacency to a metro area.

In [188]:
df_education = read_excel(dir_path + "data/Education.xls")
df_education.columns = standard_name_cols(df_education.columns)
df_education[["STATE", "AREA_NAME"]] = upper_consistent(df_education[["STATE", "AREA_NAME"]])
df_education["AREA_NAME"] = df_education["AREA_NAME"].replace("LOUSIANA", "LOUISIANA")


display(df_education.head())
check_nan(df_education)
check_duplicates(df_education, df_education.columns)


========== File name README.md

Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2019


,FIPS_CODE,STATE,AREA_NAME,2003_RURAL_URBAN_CONTINUUM_CODE,2003_URBAN_INFLUENCE_CODE,2013_RURAL_URBAN_CONTINUUM_CODE,2013_URBAN_INFLUENCE_CODE,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19
0,00000,US,UNITED STATES,NaN,NaN,NaN,NaN,52373312,34158051,11650730,...,27.4,24.4,26472261,59472748,63756905,70920162,11.998918,26.956844,28.898697,32.145542
1,01000,AL,ALABAMA,NaN,NaN,NaN,NaN,1062306,468269,136287,...,25.9,19,458922,1022839,993344,845772,13.819302,30.800268,29.912098,25.468332
2,01001,AL,AUTAUGA COUNTY,2,2,2,2,6611,3757,933,...,26.9,18,4291,12551,10596,9929,11.483395,33.588459,28.356571,26.571573
3,01003,AL,BALDWIN COUNTY,4,5,3,2,18726,8426,2334,...,29.3,23.1,13893,41797,47274,48148,9.193843,27.659616,31.284081,31.862459
4,01005,AL,BARBOUR COUNTY,6,6,6,6,8120,2242,581,...,21.3,10.9,4812,6396,4676,2080,26.786907,35.604542,26.029837,11.578713


=== Check NAN
... nb of columns with NAN 44 out of 47
LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970... 2.95%
HIGH_SCHOOL_DIPLOMA_ONLY__1970... 2.95%
SOME_COLLEGE__1_3_YEARS___1970... 2.95%
FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1970... 2.95%
PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970... 2.95%
PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__1970... 2.95%
PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE__1_3_YEARS___1970... 2.95%
PERCENT_OF_ADULTS_COMPLETING_FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1970... 2.95%
2003_RURAL_URBAN_CONTINUUM_CODE... 1.89%
2013_RURAL_URBAN_CONTINUUM_CODE... 1.89%
2013_URBAN_INFLUENCE_CODE... 1.89%
2003_URBAN_INFLUENCE_CODE... 1.89%
PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1980... 0.49%
PERCENT_OF_ADULTS_COMPLETING_FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1980... 0.49%
PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE__1_3_YEARS___1980... 0.49%
PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__1980... 0.49%
LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1980... 0.49%
FOUR_YEARS_OF

,FIPS_CODE,STATE,AREA_NAME,2003_RURAL_URBAN_CONTINUUM_CODE,2003_URBAN_INFLUENCE_CODE,2013_RURAL_URBAN_CONTINUUM_CODE,2013_URBAN_INFLUENCE_CODE,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19


50

In [185]:
state_list

array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA',
       'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA',
       'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA',
       'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE',
       'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

In [187]:
df_education[df_education["AREA_NAME"]=="LOUSIANA"]

,FIPS_CODE,STATE,AREA_NAME,2003_RURAL_URBAN_CONTINUUM_CODE,2003_URBAN_INFLUENCE_CODE,2013_RURAL_URBAN_CONTINUUM_CODE,2013_URBAN_INFLUENCE_CODE,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19
1139,22000,LA,LOUSIANA,NaN,NaN,NaN,NaN,1046551,446400,153171,...,23.7,18.7,461706,1061388,848474,753585,14.773869,33.962753,27.149839,24.113539


In [189]:
# split into states and countys

## States
df_education_state = df_education[df_education["AREA_NAME"].isin(state_list)]
states_found = df_education_state["AREA_NAME"].unique()
print(f"Nb of states found: {len(states_found)} / {len(state_list)}")

if len(states_found) != len(state_list):
    state_n_found = [state for state in state_list if state not in states_found]
    print(f"Missing state: {state_n_found}")

# nan check : columns with NAN correspond to the urban code
check_nan(df_education_state)
# drop duplicates
df_education_state = df_education_state.drop_duplicates(["STATE", "AREA_NAME"])

# keep only not null columns
df_education_state_n_nan = df_education_state.dropna(axis=1, how='all')
display(df_education_state_n_nan.head())



Nb of states found: 51 / 51
=== Check NAN
... nb of columns with NAN 4 out of 47
2003_RURAL_URBAN_CONTINUUM_CODE... 98.08%
2003_URBAN_INFLUENCE_CODE... 98.08%
2013_RURAL_URBAN_CONTINUUM_CODE... 98.08%
2013_URBAN_INFLUENCE_CODE... 98.08%


,FIPS_CODE,STATE,AREA_NAME,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1970,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__1970,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19
1,01000,AL,ALABAMA,1062306,468269,136287,141936,58.7,25.9,7.5,...,25.9,19,458922,1022839,993344,845772,13.819302,30.800268,29.912098,25.468332
69,02000,AK,ALASKA,44994,50820,20052,19082,33.3,37.7,14.9,...,35.7,24.7,34376,134582,169609,142019,7.152934,28.003729,35.292122,29.551214
106,04000,AZ,ARIZONA,383873,289781,127063,115020,41.9,31.6,13.9,...,33.1,23.5,608637,1129129,1600240,1394526,12.860705,23.858877,33.81361,29.466806
122,05000,AR,ARKANSAS,635149,271150,80835,70378,60.1,25.6,7.6,...,24.5,16.7,270168,684659,593576,463236,13.430243,34.034885,29.507084,23.02779
198,06000,CA,CALIFORNIA,4062367,3566578,1784772,1462266,37.4,32.8,16.4,...,30,26.6,4418675,5423462,7648680,8980726,16.692171,20.487896,28.89397,33.925964


In [191]:
## county
df_education_county = df_education[df_education["FIPS_CODE"].isin(county_fips_list)]
# Check if all county were found
county_found = df_education_county["FIPS_CODE"].unique()
print(f"Nb of states found: {len(county_found)} / {len(county_fips_list)}")

if len(county_found) != len(county_fips_list):
    county_n_found = [county for county in county_fips_list if county not in county_found]
    print(f"Missing state: {county_n_found}")

# TODO: fix countys
display(df_education_county.head())
check_nan(df_education_county)


# drop duplicates
df_education_county = df_education_county.drop_duplicates(["STATE", "AREA_NAME"])

Nb of states found: 3112 / 3152
Missing state: ['02901', '02902', '02903', '02904', '02905', '02906', '02907', '02908', '02909', '02910', '02911', '02912', '02913', '02914', '02915', '02916', '02917', '02918', '02919', '02920', '02921', '02922', '02923', '02924', '02925', '02926', '02927', '02928', '02929', '02930', '02931', '02932', '02933', '02934', '02935', '02936', '02937', '02938', '02939', '02940']


,FIPS_CODE,STATE,AREA_NAME,2003_RURAL_URBAN_CONTINUUM_CODE,2003_URBAN_INFLUENCE_CODE,2013_RURAL_URBAN_CONTINUUM_CODE,2013_URBAN_INFLUENCE_CODE,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19
2,01001,AL,AUTAUGA COUNTY,2,2,2,2,6611,3757,933,...,26.9,18,4291,12551,10596,9929,11.483395,33.588459,28.356571,26.571573
3,01003,AL,BALDWIN COUNTY,4,5,3,2,18726,8426,2334,...,29.3,23.1,13893,41797,47274,48148,9.193843,27.659616,31.284081,31.862459
4,01005,AL,BARBOUR COUNTY,6,6,6,6,8120,2242,581,...,21.3,10.9,4812,6396,4676,2080,26.786907,35.604542,26.029837,11.578713
5,01007,AL,BIBB COUNTY,1,1,1,1,5272,1402,238,...,20.4,7.1,3386,7256,3848,1678,20.942602,44.878773,23.800098,10.378526
6,01009,AL,BLOUNT COUNTY,1,1,1,1,10677,3440,626,...,24.8,9.6,7763,13299,13519,5210,19.509438,33.422131,33.975021,13.093413


=== Check NAN
... nb of columns with NAN 32 out of 47
LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970... 0.19%
SOME_COLLEGE__1_3_YEARS___1970... 0.19%
FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1970... 0.19%
PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970... 0.19%
PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__1970... 0.19%
PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE__1_3_YEARS___1970... 0.19%
PERCENT_OF_ADULTS_COMPLETING_FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1970... 0.19%
HIGH_SCHOOL_DIPLOMA_ONLY__1970... 0.19%
PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1980... 0.10%
PERCENT_OF_ADULTS_COMPLETING_FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1980... 0.10%
PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE__1_3_YEARS___1980... 0.10%
PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__1980... 0.10%
FOUR_YEARS_OF_COLLEGE_OR_HIGHER__1980... 0.10%
SOME_COLLEGE__1_3_YEARS___1980... 0.10%
HIGH_SCHOOL_DIPLOMA_ONLY__1980... 0.10%
LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1980... 0.10%
LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2000.

,FIPS_CODE,STATE,AREA_NAME,2003_RURAL_URBAN_CONTINUUM_CODE,2003_URBAN_INFLUENCE_CODE,2013_RURAL_URBAN_CONTINUUM_CODE,2013_URBAN_INFLUENCE_CODE,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19


In [141]:
county_fips_list

array(['01001', '01003', '01005', ..., '56041', '56043', '56045'],
      dtype=object)

In [143]:
df_education

,FIPS_CODE,STATE,AREA_NAME,2003_RURAL_URBAN_CONTINUUM_CODE,2003_URBAN_INFLUENCE_CODE,2013_RURAL_URBAN_CONTINUUM_CODE,2013_URBAN_INFLUENCE_CODE,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__1970,HIGH_SCHOOL_DIPLOMA_ONLY__1970,SOME_COLLEGE__1_3_YEARS___1970,...,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2000,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2000,LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,BACHELOR_S_DEGREE_OR_HIGHER__2015_19,PERCENT_OF_ADULTS_WITH_LESS_THAN_A_HIGH_SCHOOL_DIPLOMA__2015_19,PERCENT_OF_ADULTS_WITH_A_HIGH_SCHOOL_DIPLOMA_ONLY__2015_19,PERCENT_OF_ADULTS_COMPLETING_SOME_COLLEGE_OR_ASSOCIATE_S_DEGREE__2015_19,PERCENT_OF_ADULTS_WITH_A_BACHELOR_S_DEGREE_OR_HIGHER__2015_19
0,00000,US,UNITED STATES,NaN,NaN,NaN,NaN,52373312,34158051,11650730,...,27.4,24.4,26472261,59472748,63756905,70920162,11.998918,26.956844,28.898697,32.145542
1,01000,AL,ALABAMA,NaN,NaN,NaN,NaN,1062306,468269,136287,...,25.9,19,458922,1022839,993344,845772,13.819302,30.800268,29.912098,25.468332
2,01001,AL,AUTAUGA COUNTY,2,2,2,2,6611,3757,933,...,26.9,18,4291,12551,10596,9929,11.483395,33.588459,28.356571,26.571573
3,01003,AL,BALDWIN COUNTY,4,5,3,2,18726,8426,2334,...,29.3,23.1,13893,41797,47274,48148,9.193843,27.659616,31.284081,31.862459
4,01005,AL,BARBOUR COUNTY,6,6,6,6,8120,2242,581,...,21.3,10.9,4812,6396,4676,2080,26.786907,35.604542,26.029837,11.578713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,72145,PR,VEGA BAJA MUNICIPIO,1,1,1,1,NaN,NaN,NaN,...,21.4,14.4,10494,9681,8905,7834,28.428238,26.225822,24.123638,21.2223
3279,72147,PR,VIEQUES MUNICIPIO,7,12,7,12,NaN,NaN,NaN,...,13.7,10.1,1792,2440,875,1121,28.773281,39.177906,14.049454,17.999357
3280,72149,PR,VILLALBA MUNICIPIO,2,2,2,2,NaN,NaN,NaN,...,18,12.8,3330,5809,2987,3015,21.993263,38.366028,19.727892,19.912819
3281,72151,PR,YABUCOA MUNICIPIO,1,1,1,1,NaN,NaN,NaN,...,18.3,12.1,6927,6132,6494,4293,29.048897,25.715004,27.233078,18.003019



========== File name README.md

Educational attainment for adults age 25 and older for the U.S., States, and counties, 1970-2019
                                                   Unnamed: 1     Unnamed: 2  \
Educational attainment for adults age 25 and ol...                             
Sources: U.S. Census Bureau, 1970, 1980, 1990, ...        NaN            NaN   
For definitions of rural classifications, see t...        NaN            NaN   
This table was prepared by USDA, Economic Resea...        NaN            NaN   
FIPS Code                                               State      Area name   
00000                                                      US  United States   

                                                                         Unnamed: 3  \
Educational attainment for adults age 25 and ol...                                    
Sources: U.S. Census Bureau, 1970, 1980, 1990, ...                              NaN   
For definitions of rural classifications, see t.

FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970","Four years of college or higher, 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2015-19","High school diploma only, 2015-19","Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19","Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
0,US,United States,NaN,NaN,NaN,NaN,52373312,34158051,11650730,11717266,...,27.4,24.4,26472261,59472748,63756905,70920162,11.998918,26.956844,28.898697,32.145542
1,AL,Alabama,NaN,NaN,NaN,NaN,1062306,468269,136287,141936,...,25.9,19,458922,1022839,993344,845772,13.819302,30.800268,29.912098,25.468332
2,AL,Autauga County,2,2,2,2,6611,3757,933,767,...,26.9,18,4291,12551,10596,9929,11.483395,33.588459,28.356571,26.571573
3,AL,Baldwin County,4,5,3,2,18726,8426,2334,2038,...,29.3,23.1,13893,41797,47274,48148,9.193843,27.659616,31.284081,31.862459
4,AL,Barbour County,6,6,6,6,8120,2242,581,861,...,21.3,10.9,4812,6396,4676,2080,26.786907,35.604542,26.029837,11.578713


FIPS Code,index,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2015-19","High school diploma only, 2015-19","Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19","Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
0,0,US,United States,NaN,NaN,NaN,NaN,52373312,34158051,11650730,...,27.4,24.4,26472261,59472748,63756905,70920162,11.998918,26.956844,28.898697,32.145542
1,1,AL,Alabama,NaN,NaN,NaN,NaN,1062306,468269,136287,...,25.9,19,458922,1022839,993344,845772,13.819302,30.800268,29.912098,25.468332
2,2,AL,Autauga County,2,2,2,2,6611,3757,933,...,26.9,18,4291,12551,10596,9929,11.483395,33.588459,28.356571,26.571573
3,3,AL,Baldwin County,4,5,3,2,18726,8426,2334,...,29.3,23.1,13893,41797,47274,48148,9.193843,27.659616,31.284081,31.862459
4,4,AL,Barbour County,6,6,6,6,8120,2242,581,...,21.3,10.9,4812,6396,4676,2080,26.786907,35.604542,26.029837,11.578713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,3278,PR,Vega Baja Municipio,1,1,1,1,NaN,NaN,NaN,...,21.4,14.4,10494,9681,8905,7834,28.428238,26.225822,24.123638,21.2223
3279,3279,PR,Vieques Municipio,7,12,7,12,NaN,NaN,NaN,...,13.7,10.1,1792,2440,875,1121,28.773281,39.177906,14.049454,17.999357
3280,3280,PR,Villalba Municipio,2,2,2,2,NaN,NaN,NaN,...,18,12.8,3330,5809,2987,3015,21.993263,38.366028,19.727892,19.912819
3281,3281,PR,Yabucoa Municipio,1,1,1,1,NaN,NaN,NaN,...,18.3,12.1,6927,6132,6494,4293,29.048897,25.715004,27.233078,18.003019


In [21]:
df_education.columns

Index(['State', 'Area name', '2003 Rural-urban Continuum Code',
       '2003 Urban Influence Code', '2013 Rural-urban Continuum Code',
       '2013 Urban Influence Code', 'Less than a high school diploma, 1970',
       'High school diploma only, 1970', 'Some college (1-3 years), 1970',
       'Four years of college or higher, 1970',
       'Percent of adults with less than a high school diploma, 1970',
       'Percent of adults with a high school diploma only, 1970',
       'Percent of adults completing some college (1-3 years), 1970',
       'Percent of adults completing four years of college or higher, 1970',
       'Less than a high school diploma, 1980',
       'High school diploma only, 1980', 'Some college (1-3 years), 1980',
       'Four years of college or higher, 1980',
       'Percent of adults with less than a high school diploma, 1980',
       'Percent of adults with a high school diploma only, 1980',
       'Percent of adults completing some college (1-3 years), 1980',
   